In [3]:
# AR
# 12.2019

In [4]:
# http://theo.cs.ovgu.de/lehre/lehre2019w/reasoning/05-uebungsblatt.pdf

In [5]:
from gavel.selection.selector import Selector, Sine

In [6]:
from gavel.logic.logic import BinaryFormula, Constant, PredicateExpression, BinaryConnective, FunctorExpression, DefinedConstant

In [7]:
from gavel.prover.base.interface import BaseProverInterface

In [8]:
from gavel.logic.proof import Proof
from gavel.logic.proof import ProofStep

In [9]:
from gavel.dialects.tptp.parser import TPTPProblemParser

problem_parser = TPTPProblemParser()

In [10]:
# Convert string to gavel.logic.problem.Problem
def read_prob(st):
    x = problem_parser.parse(st)
    return next(x)

In [11]:
p1 = read_prob(open('P4/problem1.p', 'r').readlines())
p2 = read_prob(open('P4/problem2.p', 'r').readlines())
p3 = read_prob(open('P4/problem3.p', 'r').readlines())
p4 = read_prob(open('P4/problem4.p', 'r').readlines())

p12 = read_prob(open('P5/problem1.p', 'r').readlines())
p22 = read_prob(open('P5/problem2.p', 'r').readlines())
p32 = read_prob(open('P5/problem3.p', 'r').readlines())

In [12]:
# Convert premises into list of strings
def formulas(p):
    if type(p)==list:
        return [str(i.formula) for i in p]
    return str(p.formula)

In [13]:
formulas(p32.premises)

['((p(e)) & ((p(b)) & (p(d)))) => ($false)',
 '(p(e)) => (p(d))',
 '($true) => (p(f))',
 '(p(a)) => ($false)',
 '(p(c)) => (p(e))',
 '($true) => (p(c))',
 '($true) => ((f) = (b))']

In [14]:
import time
# print how long the function took to execute
def timer(func):
    """Print the runtime of the decorated function"""
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

In [15]:
class Horn(BaseProverInterface):
    def _submit_problem(self, problem_instance, *args, **kwargs):
        # Call your prov:qer here
        sol = HornSolver()
        result = sol.prove(problem_instance)
        return result

In [16]:
class HornV2(BaseProverInterface):
    def _submit_problem(self, problem_instance, *args, **kwargs):
        # Call your prov:qer here
        sol = Solver()
        result = sol.prove(problem_instance)
        return result

In [17]:
ATOMS = [Constant, DefinedConstant, str]

In [18]:
def st(x):
    if type(x) in [str, DefinedConstant]:
        return str(x)
    return str(x.symbol)

In [763]:
## Alternative solver class with actual ProofStep Objects. This allows us to merge the old/new variables
class Solver():
    def __init__(self):
        self.equiv = {}
        self.func = {}
        self.S = {'$true'}
        self.preds = {}
        self.G = 0
        
        self.steps = [ProofStep(DefinedConstant.VERUM)]
        
    def functor(self, f):
        if type(f) in ATOMS:
            return st(f)
        
        if all([type(i) in ATOMS for i in f.arguments]):
            if str(f) not in self.func:
                k = f'K{self.G}'
                self.func[str(f)] = k
                self.equiv[k] = set([k])
                self.G += 1
                return k
            return self.func[str(f)]

        l = [self.functor(i) for i in f.arguments]

        return self.functor(FunctorExpression(f.functor, l))
    
    def eq(self, x, y): 
        x = self.functor(x)
        y = self.functor(y)
        
        #self.steps[-1].add(f"{x}={y}")
        
        if x not in self.equiv[y]:
            self.equiv[x] = self.equiv[y]
            self.equiv[x].add(x)
        
    def check(self, exp):
        if type(exp) in [Constant, DefinedConstant]:
            return str(exp) in self.S
        
        if type(exp) == FunctorExpression:
            return self.functor(exp) in self.S
        
        elif type(exp) == PredicateExpression:
            p = str(exp.predicate)
            if p in self.preds:
                v = tuple(self.functor(i) for i in exp.arguments)
                for x in self.preds[p]:
                    assert len(v) == len(x), f'Non matching lengths. v: {v}, x: {x}, p:{p}'
                    if all(v[i] in self.equiv[x[i]] for i in range(len(v))):
                        return True
            return False
        
        elif type(exp) == BinaryFormula:
            return self.check(exp.left) and self.check(exp.right) # Only supports and for the time being
        print('?')
        print(type(exp))
                 
    def prepare(self, exp):
        if type(exp) == Constant:
            k = str(exp.symbol)
            if k not in self.equiv:
                self.equiv[k] = set(str(exp))
            return
        if type(exp) == BinaryFormula:
            self.prepare(exp.left)
            self.prepare(exp.right)
            return
        if type(exp) in (PredicateExpression, FunctorExpression):
            for i in exp.arguments:
                self.prepare(i)
        
            
    def prove(self, problem, *args, **kwargs):
        lst = [i.formula for i in problem.premises]
        for exp in lst:
            self.prepare(exp)

        conj = problem.conjecture.formula
        
        update = True
        while update:
            update = False
            
            for exp in lst:
                r = exp.right
                
                # We already know
                if type(r) == BinaryFormula and r.operator == BinaryConnective.EQ:
                    x = self.functor(r.left)
                    y = self.functor(r.right)
                    if x in self.equiv[y]:
                        continue
                elif type(r) == DefinedConstant and str(r) in self.S:
                    continue
                elif type(r) == PredicateExpression:
                    p = str(r.predicate)
                    if p in self.preds:
                        v = tuple(self.functor(i) for i in r.arguments)
                        match = False
                        for x in self.preds[p]:
                            if all(v[i] in self.equiv[x[i]] for i in range(len(v))):
                                match = True
                                break
                        if match:
                            continue
                                
                
                # We don't know
                if self.check(exp.left):
                    update = True
                    
                    if type(r) == BinaryFormula and r.operator == BinaryConnective.EQ:
                        self.eq(r.left, r.right)

                    elif type(r) == DefinedConstant:
                        self.S.add(str(r))

                    elif type(r) == PredicateExpression:
                        self.S.add(str(r))
                        p = str(r.predicate)
                        if p not in self.preds:
                            self.preds[p] = set()
                        v = tuple(self.functor(i) for i in r.arguments)
                        self.preds[p].add(v)
                    else:
                        print("???")
                        print(type(r))
                    self.steps.append(ProofStep(exp))
                    if type(conj) == type(r) and conj == r:
                        return Proof(steps=self.steps)
        

In [764]:
class HornSolver():
    def __init__(self):
        self.oldEq = {}
        self.equiv = {}
        self.func = {}
        self.oldS = set()
        self.S = {'$true'}
        self.oldPreds = {}
        self.preds = {}
        self.G = 0
        
        self.steps = [{'$true'}]
        
    def functor(self, f):
        if type(f) in ATOMS:
            return st(f)
        
        if all([type(i) in ATOMS for i in f.arguments]):
            if str(f) not in self.func:
                k = f'K{self.G}'
                self.func[str(f)] = k
                self.equiv[k] = set([k])
                self.G += 1
                return k
            return self.func[str(f)]

        l = [self.functor(i) for i in f.arguments]

        return self.functor(FunctorExpression(f.functor, l))
    
    def eq(self, x, y): 
        if type(x) == FunctorExpression:
            k = self.functor(x)
            
            
        x = self.functor(x)
        y = self.functor(y)

        if x not in self.equiv[y]:
            self.equiv[x] = self.equiv[y]
            self.equiv[x].add(x)
            
        
    def check(self, exp):
        if type(exp) in [Constant, DefinedConstant]:
            return str(exp) in self.oldS
        
        if type(exp) == FunctorExpression:
            return self.functor(exp) in self.oldS
        
        elif type(exp) == PredicateExpression:
            p = str(exp.predicate)
            if p in self.oldPreds:
                v = tuple(self.functor(i) for i in exp.arguments)
                for x in self.oldPreds[p]:
                    assert len(v) == len(x), f'Non matching lengths. v: {v}, x: {x}, p:{p}'
                    if all(v[i] in self.oldEq[x[i]] for i in range(len(v))):
                        return True
            return False
        
        elif type(exp) == BinaryFormula:
            return self.check(exp.left) and self.check(exp.right) # Only supports and for the time being
        print('?')
        print(type(exp))
                 
    def prepare(self, exp):
        if type(exp) == Constant:
            k = str(exp.symbol)
            if k not in self.equiv:
                self.equiv[k] = set(str(exp))
            return
        if type(exp) == BinaryFormula:
            self.prepare(exp.left)
            self.prepare(exp.right)
            return
        if type(exp) in (PredicateExpression, FunctorExpression):
            for i in exp.arguments:
                self.prepare(i)
        
            
    def prove(self, problem, *args, **kwargs):
        lst = [i.formula for i in problem.premises]
        for exp in lst:
            self.prepare(exp)

        conj = problem.conjecture.formula
        
        neweq = False
        while self.S != self.oldS or neweq:
            neweq = False
            self.steps.append(set())
            self.oldS = self.S.copy()
            self.oldEq = self.equiv.copy()
            self.oldPreds = self.preds.copy()
            
            for exp in lst:
                r = exp.right
                
                # We already know
                if type(r) == BinaryFormula and r.operator == BinaryConnective.EQ:
                    x = self.functor(r.left)
                    y = self.functor(r.right)
                    if x in self.equiv[y]:
                        continue
                elif type(r) == DefinedConstant and str(r) in self.S:
                    continue
                elif type(r) == PredicateExpression:
                    p = str(r.predicate)
                    if p in self.preds:
                        v = tuple(self.functor(i) for i in r.arguments)
                        for x in self.preds[p]:
                            assert len(v) == len(x), f'Non matching lengths. v: {v}, x: {x}, p:{p}'
                            if all(v[i] in self.equiv[x[i]] for i in range(len(v))):
                                continue
                
                # We don't know
                if self.check(exp.left):
                    
                    if type(r) == BinaryFormula and r.operator == BinaryConnective.EQ:
                        self.steps[-1].update({str(r)})
                        self.eq(r.left, r.right)
                        neweq = True

                    elif type(r) == DefinedConstant:
                        self.S.add(str(r))

                    elif type(r) == PredicateExpression:
                        self.S.add(str(r))
                        p = str(r.predicate)
                        if p not in self.preds:
                            self.preds[p] = set()
                        v = tuple(self.functor(i) for i in r.arguments)
                        self.preds[p].add(v)
                    else:
                        print("???")
                        print(type(r))
                    if type(conj) == type(r) and conj == r:
                        self.steps[-1].update(self.S - self.oldS)
                        return Proof(steps=self.steps)
            self.steps[-1].update(self.S - self.oldS)

In [765]:
hop = Horn()
h2 = HornV2()

In [785]:
@timer
def prove(prob, prnt=False):
    if prnt:
        print("\nPremises")
        print(formulas(prob.premises))
    print(f"\nConjecture: {str(prob.conjecture.formula)}")
    p = hop.prove(prob)
    if p:
        print("Proof found!")
        if prnt:
            for i in p.steps:
                print(i)
        else:
            print(f'Steps required: {len(p.steps)}')
        return p
    else:
        print("No proof found")
    print('\n')

@timer
def prove2(prob, prnt=False):
    if prnt:
        print("Premises")
        print(formulas(prob.premises))
    print(f"\nConjecture: {str(prob.conjecture.formula)}")
    p = h2.prove(prob)
    if p:
        print("Proof found!")
        if prnt:
            for i in p.steps:
                print(i)
        else:
            print(f'Steps required: {len(p.steps)}')
        return p
    else:
        print("No proof found")
    print('\n')

In [791]:
prove(p22, True)


Premises
['($true) => ((a) = (d))', '($true) => (p(g(f(a), b)))', '($true) => ((f(d)) = (b))', '($true) => ((g(b, b)) = (c))', '(p(c)) => ($false)']

Conjecture: $false
No proof found


Finished 'prove' in 0.0005 secs


In [788]:
for prob in [p1, p2, p3, p4, p12, p22, p32]:
    prove(prob, True)


Premises
['(p(a)) => (q(a))', '(q(a)) => ($false)', '($true) => (p(a))']

Conjecture: $false
Proof found!
{'$true'}
{'p(a)'}
{'q(a)'}
{'$false'}
Finished 'prove' in 0.0008 secs

Premises
['((p(a)) & ((p(b)) & (p(d)))) => ($false)', '(p(e)) => ($false)', '(p(c)) => (p(a))', '(p(c)) => ($false)', '(p(a)) => (p(d))']

Conjecture: $false
No proof found


Finished 'prove' in 0.0005 secs

Premises
['((p(e)) & ((p(b)) & (p(d)))) => ($false)', '(p(e)) => (p(d))', '($true) => (p(f))', '(p(a)) => ($false)', '(p(c)) => (p(e))', '($true) => (p(c))']

Conjecture: $false
No proof found


Finished 'prove' in 0.0009 secs

Premises
['((p(e)) & ((p(b)) & (p(d)))) => ($false)', '(p(e)) => (p(d))', '($true) => (p(b))', '(p(a)) => ($false)', '(p(c)) => (p(e))', '($true) => (p(c))']

Conjecture: $false
Proof found!
{'$true'}
{'p(b)', 'p(c)'}
{'p(e)'}
{'p(d)'}
{'$false'}
Finished 'prove' in 0.0009 secs

Premises
['($true) => ((a) = (b))', '($true) => (p(a))', '(p(b)) => ($false)']

Conjecture: $false
Proof 

In [758]:
big = read_prob(open('big.p').readlines())

In [27]:
from gavel.logic.logic import LogicElement
from itertools import chain
from typing import List, Dict
from gavel.logic.problem import Problem

In [28]:
def occurrences(prob): # O(n)
    total = {}
    pos = {}
    for i, p in enumerate(prob.premises):
        s = appear(p.formula)
        for x in s:
            if x in total:
                total[x] += 1
                pos[x].add(i)
            else:
                total[x] = 1
                pos[x] = {i}
            
    return total#, pos

def appear(f): # O(k), where k is the average ammount of symbols per premise (so constant)
    if type(f) in ATOMS:
        return {st(f)}
    
    if type(f) == BinaryFormula:
        s = appear(f.left)
        s.update(appear(f.right))
        return s
    #print(type(f))
    if type(f) == FunctorExpression:
        s = {str(f.functor)}
    else:
        s = {str(f.predicate)}
    for i in f.arguments:
        s.update(appear(i))
    return s

def trigger_dict(prob, occ, tolerance, gen_thresh): #O(n)
    t = {}
    for i, p in enumerate(prob.premises):
        s = appear(p.formula)
        for x in s:
            if x not in t:
                t[x] = set()
            if occ[x] <= gen_thresh or all([occ[x] <= tolerance*occ[y] for y in s-{x}]):
                t[x].add(i)
    return t

In [29]:
arst = None
class Sine(Selector): # O(n) + O(n) = O(n)
    def select(self, problem, tolerance=1, depth=None, gen_thresh=1):
        occ = occurrences(problem) # O(n)
        t = trigger_dict(problem, occ, tolerance, gen_thresh)
        s = {}
        return Problem(premises=self.calculate_triggers(problem, t, depth), conjecture=problem.conjecture), t, occ

    def calculate_triggers(self, problem, trig, depth):
        k_triggered = appear(problem.conjecture.formula)
        remaining = problem.premises.copy()
        used = set()
        k = 0
        ans = set()
        while k_triggered and (depth is None or k < depth):
            k += 1
            untriggered = []
            new = set()
            for i, p in enumerate(remaining):
                s = appear(p.formula)
                if any(i in trig[x] for x in k_triggered):
                    #print(f'i: {i}, s: {s}')
                    new.update(appear(p.formula))
                    ans.add(i)
                else:
                    untriggered.append(p)
            remaining = untriggered.copy()
            used.update(k_triggered)
            k_triggered = new-used
        global arst
        arst = untriggered
        return [problem.premises[i] for i in ans]


In [30]:
s = Sine()

In [31]:
prob, t, occ = s.select(big, depth=2, gen_thresh=1000)

In [32]:
t['q']

set()

In [33]:
formulas(prob.premises)

['((p(e)) & ((p(b)) & (p(d)))) => ($false)',
 '(p(e)) => (p(d))',
 '($true) => (p(b))',
 '(p(a)) => ($false)']

In [34]:
str(arst[0].formula.left.predicate)

'p'

In [35]:
all('x1' in appear(i.formula) for i in arst)

False

In [36]:
formulas(big.premises[10005])

'($true) => (p(c))'

In [37]:
len(prob.premises)

4

In [38]:
prove(prob)


Conjecture: $false
No proof found


Finished 'prove' in 0.0009 secs


In [39]:
prove2(big)


Conjecture: $false
No proof found


Finished 'prove2' in 0.0003 secs


In [40]:
formulas(big.premises[-10:])

['($true) => (q(x1, x3))',
 '(q(z5, z3)) => (q(f(z3, x0), y9))',
 '($true) => (q(y1, y2))',
 '(q(x5, x1)) => (p(z5))',
 '((p(e)) & ((p(b)) & (p(d)))) => ($false)',
 '(p(e)) => (p(d))',
 '($true) => (p(b))',
 '(p(a)) => ($false)',
 '(p(c)) => (p(e))',
 '($true) => (p(c))']

In [670]:
from itertools import chain
from typing import List, Dict

from gavel.logic.logic import LogicElement
from gavel.logic.problem import Problem

In [658]:
all(i.role for i in p2.premises)

True

In [220]:
class Sine(Selector):
    
    def select(self, problem, max_depth=10):
        symbols = set(
            chain(smbls(problem.conjecture), *(smbls(p) for p in problem.premises))
        )
        premise_symbols = {p.name: set(smbls(p)) for p in problem.premises}
        commonness = {
            s: sum(1 for ps in premise_symbols.values() if s in ps)
            for s in symbols
        }
        
        print(symbols)
        print(premise_symbols)

        return Problem(
            premises=list(self.calculate_triggers(problem, premise_symbols, commonness, max_depth)),
            conjecture=problem.conjecture)

    def trigger(self, symbol, sentence: LogicElement, commonness: Dict[str, int]) -> bool:
        return symbol in smbls(sentence) and all(
            commonness[symbol] <= commonness[symbol2]
            for symbol2 in sentence.symbols()
        )

    def calculate_triggers(self, problem: Problem, premise_symbols, commonness, max_depth):
        remaining_premises = list(problem.premises)
        k_triggered_symbols = set(smbls(problem.conjecture))
        used_symbols = set()
        k = 0
        while k_triggered_symbols and k < max_depth:
            k += 1
            untriggered_premises = []
            newer_symbols = set()
            for p in remaining_premises:
                p_symbs = premise_symbols[p.name]
                # If s is k-step triggered and s triggers A, then A is k + 1-step triggered
                if any(self.trigger(s, p, commonness) for s in k_triggered_symbols):
                    newer_symbols = newer_symbols.union(p_symbs)
                    yield p
                else:
                    untriggered_premises.append(p)
            remaining_premises = untriggered_premises
            used_symbols = used_symbols.union(k_triggered_symbols)
            k_triggered_symbols = newer_symbols.difference(used_symbols)

In [226]:
def smbls(p):
    try:
        return p.symbols()
    except:
        print(p.name)
        return p.name

In [227]:
fs = [str(p.formula) for p in p2.premises]

In [228]:
fs

['((p(a)) & ((p(b)) & (p(d)))) => ($false)',
 '(p(e)) => ($false)',
 '(p(c)) => (p(a))',
 '(p(c)) => ($false)',
 '(p(a)) => (p(d))']

In [229]:
selector = Sine()
small_problem = selector.select(p2)
hop.prove(small_problem)

TypeError: 'AnnotatedFormula' object is not iterable

In [206]:
small_problem.premises

[]

In [207]:
p2.premises

In [51]:
prove(big)

Finished 'prove' in 20.8934 secs


In [52]:
prove2(big)

Finished 'prove2' in 20.6628 secs


In [53]:
prove(big)

Finished 'prove' in 20.4444 secs


In [54]:
prove2(big)

Finished 'prove2' in 20.6582 secs


In [42]:
print(big.conjecture.formula)

$false


In [ ]:
big.conjecture

In [43]:
formulas(big.premises[-10:])

['($true) => (q(x1, x3))',
 '(q(z5, z3)) => (q(f(z3, x0), y9))',
 '($true) => (q(y1, y2))',
 '(q(x5, x1)) => (p(z5))',
 '((p(e)) & ((p(b)) & (p(d)))) => ($false)',
 '(p(e)) => (p(d))',
 '($true) => (p(b))',
 '(p(a)) => ($false)',
 '(p(c)) => (p(e))',
 '($true) => (p(c))']

In [580]:
class dumb():
    def __init__(self, i):
        self.formula = i

In [857]:
class btSelector():
    def __init__(self):
        self.func = {}
        self.equiv = {}
        self.G = 0
        self.remaining = None
        self.premises = []
        self.relevant = None
        self.preds = {}
        
    def prepare(self, exp):
        if type(exp) == Constant:
            k = str(exp.symbol)
            if k not in self.equiv:
                self.equiv[k] = set(str(exp))
            return
        if type(exp) == DefinedConstant:
            k = str(exp)
            if k not in self.equiv:
                self.equiv[k] = set([k])
            return
        if type(exp) == BinaryFormula:
            self.prepare(exp.left)
            self.prepare(exp.right)
            return
        if type(exp) in (PredicateExpression, FunctorExpression):
            for i in exp.arguments:
                self.prepare(i)
    
    def flatten(self, f):
        if type(f) in ATOMS:
            return st(f)

        if all([type(i) in ATOMS for i in f.arguments]):
            
            key = str(f.functor)
            args = tuple([flatten(i) for i in f.arguments])
            
            if key not in self.func:   # new functor
                self.func[key] = {}
            
            for tup in self.func[key]: # old functor, known equivalence class
                assert len(args) == len(tup), f'Non matching lengths. functor:{key}, existing: {tup}, new: {arg_tuple}' 
                if all(args[i] in self.equiv[tup[i]] for i in range(len(tup))):
                    return self.func[key][tup]
                
            const = f'K{self.G}'       # old functor, new equivalence class
            self.func[key][args] = const
            self.equiv[const] = set([const])
            self.G += 1
            return const

        l = [self.flatten(i) for i in f.arguments]

        return self.flatten(FunctorExpression(f.functor, l))
    

    def is_useful(self, right):
        if type(right) == PredicateExpression:
            p = str(right.predicate)
            if p in self.preds:
                v = tuple(self.flatten(i) for i in right.arguments)
                for x in self.preds[p]:
                    assert len(v) == len(x), f'Non matching lengths. v: {v}, x: {x}, p:{p}' 
                    if all(v[i] in self.equiv[x[i]] for i in range(len(v))):
                        return True
                    else:
                        print(f'v: {v}, x: {x}, p:{p}, preds:{self.preds[p]}')
                        #print(v, self.preds[p])
            return False
        elif type(right) == BinaryFormula:
            if right.operator == BinaryConnective.EQ:
                return True
            elif right.operator == BinaryConnective.IMPLICATION:
                return self.is_useful(right.right)
        elif type(right) in [DefinedConstant, Constant]:
            return len(self.equiv[st(right)].intersection(self.relevant)) > 0
        else:
            print('use')
            print(type(right))

    def add_to_relevant(self, left):
        if type(left) == BinaryFormula:
            self.add_to_relevant(left.left)
            self.add_to_relevant(left.right)
        elif type(left) == PredicateExpression:
            k = str(left.predicate)
            if k not in self.preds:
                self.preds[k] = set()
            v = tuple(self.flatten(i) for i in left.arguments)
            self.preds[k].add(v)
        elif type(left) in [DefinedConstant]:
            self.relevant.add(st(left))
        else:
            print('rlv')
            print(type(left))
        
    def iter_find_relevant(self):
        rem = []
        for p in self.remaining:
            if self.is_useful(p.formula.right):
                self.premises.append(p)
                
                self.add_to_relevant(p.formula.left)
                
            else:
                rem.append(p)
        self.remaining = rem
                
    def select(self, problem):
        self.G = 0
        self.func = {}
        self.equiv = {}
        self.remaining = problem.premises
        self.premises = []
        self.relevant = {self.flatten(problem.conjecture.formula)}
        self.preds = {}
        
        for i in [p.formula for p in self.remaining]:
            self.prepare(i)
            if type(i.right) == BinaryFormula and i.right.operator == BinaryConnective.EQ:
                l = self.flatten(i.right.left)
                self.equiv[l] = self.equiv[self.flatten(i.right.right)]
                self.equiv[l].add(l)
        
        for i in range(5):
            #self.premises.append(dumb(i))
            self.iter_find_relevant()
        return self.premises

In [858]:
all(i in [0,1,2,3] for i in range(5))

False

In [859]:
all(i<10 for i in range(12))

False

In [860]:
for i in BinaryConnective:
    print(i.is_associative(), repr(i))

True &
True |
False <=>
False =>
False <=
False <~>
False ~&
False ~|
True =
False !=
False @
False *
False +
False -->
False :=
False >


In [861]:
sel = btSelector()

In [862]:
prob = p22
#prob = Problem(premises=test, conjecture=p1.conjecture)

In [863]:
ans = sel.select(prob)

In [864]:
sel.relevant

{'$false', '$true'}

In [865]:
sel.preds

{'p': {('c',)}}

In [866]:
[str(i.formula) for i in ans]

['($true) => ((a) = (d))',
 '($true) => ((f(d)) = (b))',
 '($true) => ((g(b, b)) = (c))',
 '(p(c)) => ($false)',
 '($true) => (p(g(f(a), b)))']

In [867]:
formulas(prob.premises)

['($true) => ((a) = (d))',
 '($true) => (p(g(f(a), b)))',
 '($true) => ((f(d)) = (b))',
 '($true) => ((g(b, b)) = (c))',
 '(p(c)) => ($false)']

In [868]:
sel.equiv

{'$true': {'$true'},
 'a': {'a', 'd'},
 'd': {'a', 'd'},
 'b': {'K0', 'b'},
 'K0': {'K0', 'b'},
 'c': {'K1', 'c'},
 'K1': {'K1', 'c'},
 '$false': {'$false'}}

In [869]:
sel.func

{'f': {('d',): 'K0'}, 'g': {('b', 'b'): 'K1'}}

In [258]:
formulas(p12.premises)

['($true) => ((a) = (b))', '($true) => (p(a))', '(p(b)) => ($false)']

In [259]:
p12.premises[0].formula.right.operator

=

In [150]:
class tree():
    def __init__(self, ziel):
        self.ziel = ziel

    def get(self):
        print(self.ziel)

In [151]:
t = tree(str(p1.conjecture.formula))

In [152]:
t.get()

$false
